# Sistema de recomendacion tipo Netflix con Sklearn

# Tipos de sistemas de recomendacion:

- ### Basado en polularidad:
Son relativamente faciles de implementar ya que no necesitamos algoritmos complejos, ni datos especificos del usuario. Suelen estar basados en estadistica basica, y se ofrecen las mismas sugerencias a todos los usuarios, centradose en lo que es popular entre la mayoria
  
- ### Filtrado basado en contenido
El filtrado basado en contenido se centra en las caracteristicas del item, recomendando items similares a aquellos en los que el usuario ha mostrado previo interies, siemrpre basandose en sus caracteristicas.
  

- ### Filtrado Colaborativo
Este esta enfocado en las preferencias colectivas de los usuarios. Puede ser basado en el *usuario* o *item*, en el primer caso las recomendaciones se basan en las prefetencias de usuarios similares, por otra parte en el segundo caso, las recomendaciones estan basadas en items similares a los que el usuario ha gustado anteriormente.

# Instalacion de librerias

In [1]:
%pip install tqdm==4.66.4
%pip install pandas==2.1.4
%pip install scikit-learn==1.5.1

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.neighbors import NearestNeighbors
import statistics

def warn(*args, **kwargs):
    pass

import warnings

warnings.warn = warn
warnings.filterwarnings('ignore')


# Importamos Dataset de Kaggle
El dataset se tomó de [Kaggle](https://www.kaggle.com/datasets/shubhammehta21/movie-lens-small-latest-dataset/data).ta).
Este conjunto de datos describe la actividad de calificación de 5 estrellas y etiquetado de  librelibre en MovieLens, un servicio de recomendación de películas. Los usuarios fueron seleccionados al azar para su inclusión. No se incluye información demográfica. Cada usuario está representado por un ID y no se proporciona ninguna otra información.

Los datos se encuentran en los archivos `movies.csv`, `ratings.csv` y `tags.csv`.

En el archivo `movies.csv`:

* `movieId`: ID de la película/serie (único)
* `title`: Título de la película/serie
* `genres`: Género de la película/serie

En el archivo `ratings.csv`:

* `userId`: ID del usuario que dio una calificación
* `movieId`: ID de la película/serie calificada
* `rating`: Calificación otorgada a la película/serie
* `timestamp`: Momento en que se especificó la calificación

En el archivo `tags.csv`:

* `userId`: ID del usuario que dio una calificación
* `movieId`: ID de la película/serie calificada
* `tag`: Etiquetas asignadas a la película/serie
* `timestamp`: Momento en el que se especificó la etiqueta



In [3]:
movie_df = pd.read_csv('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/BxZuF3FrO7Bdw6McwsBaBw/movies.csv')
rating_df = pd.read_csv('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/R-bYYyyf7s3IUE5rsssmMw/ratings.csv')
tag_df = pd.read_csv('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/UZKHhXSl7Ft7t9mfUFZJPQ/tags.csv')

In [4]:
movie_df.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [5]:
tag_df.head()

,userId,movieId,tag,timestamp
0,2,60756,funny,1445714994
1,2,60756,Highly quotable,1445714996
2,2,60756,will ferrell,1445714992
3,2,89774,Boxing story,1445715207
4,2,89774,MMA,1445715200


In [6]:
rating_df.sample(5)

,userId,movieId,rating,timestamp
18213,115,2325,4.0,966977847
35710,241,215,5.0,1447536409
46765,307,158,1.5,1186161857
83190,527,780,4.0,1033175078
60380,390,7371,4.5,1250341758


In [7]:
# Unimos los 3 datasets para crear uno solo con toda la informacion que necesaria
user_movie_df = movie_df.merge(rating_df, on = 'movieId', how = 'inner')
df = user_movie_df.merge(tag_df, on = ['movieId', 'userId'], how = 'inner')
df

,movieId,title,genres,userId,rating,timestamp_x,tag,timestamp_y
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,336,4.0,1122227329,pixar,1139045764
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,474,4.0,978575760,pixar,1137206825
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,567,3.5,1525286001,fun,1525286013
3,2,Jumanji (1995),Adventure|Children|Fantasy,62,4.0,1528843890,fantasy,1528843929
4,2,Jumanji (1995),Adventure|Children|Fantasy,62,4.0,1528843890,magic board game,1528843932
...,...,...,...,...,...,...,...,...
3471,187595,Solo: A Star Wars Story (2018),Action|Adventure|Children|Sci-Fi,62,4.0,1528934550,star wars,1528934552
3472,193565,Gintama: The Movie (2010),Action|Animation|Comedy|Sci-Fi,184,3.5,1537098554,anime,1537098582
3473,193565,Gintama: The Movie (2010),Action|Animation|Comedy|Sci-Fi,184,3.5,1537098554,comedy,1537098587
3474,193565,Gintama: The Movie (2010),Action|Animation|Comedy|Sci-Fi,184,3.5,1537098554,gintama,1537098603


In [8]:
# Eliminamos los campos que no necesitaremos para el analisis
df.drop(columns = ['timestamp_x', 'timestamp_y'], inplace = True)
df

,movieId,title,genres,userId,rating,tag
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,336,4.0,pixar
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,474,4.0,pixar
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,567,3.5,fun
3,2,Jumanji (1995),Adventure|Children|Fantasy,62,4.0,fantasy
4,2,Jumanji (1995),Adventure|Children|Fantasy,62,4.0,magic board game
...,...,...,...,...,...,...
3471,187595,Solo: A Star Wars Story (2018),Action|Adventure|Children|Sci-Fi,62,4.0,star wars
3472,193565,Gintama: The Movie (2010),Action|Animation|Comedy|Sci-Fi,184,3.5,anime
3473,193565,Gintama: The Movie (2010),Action|Animation|Comedy|Sci-Fi,184,3.5,comedy
3474,193565,Gintama: The Movie (2010),Action|Animation|Comedy|Sci-Fi,184,3.5,gintama


# Analisis Exploratorio de los Datos (EDA)

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3476 entries, 0 to 3475
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   movieId  3476 non-null   int64  
 1   title    3476 non-null   object 
 2   genres   3476 non-null   object 
 3   userId   3476 non-null   int64  
 4   rating   3476 non-null   float64
 5   tag      3476 non-null   object 
dtypes: float64(1), int64(2), object(3)
memory usage: 163.1+ KB


# Basado en Popularidad
La recomendación basada en popularidad recomienda artículos, en este caso, películas, basándose en lo que es popular en el sitio. Es el sistema de recomendación más básico. El sistema identifica los artículos populares considerando métricas como el número de visualizaciones, valoraciones o compras y los sugiere a todos los usuarios. En este tipo de sistema de recomendación, todos los usuarios reciben las mismas recomendaciones. El sistema puede sugerir artículos según lo que es popular en su país.

Este enfoque garantiza que los usuarios estén al tanto del contenido popular actual, lo cual puede ser útil para los nuevos usuarios que aún no tienen un historial de visualización en la plataforma. Sin embargo, esto también supone una limitación, ya que todos reciben las mismas sugerencias, que pueden no ser siempre relevantes o interesantes para ellos. Esta falta de especificidad puede resultar en una experiencia de usuario menos atractiva en comparación con sistemas de recomendación más personalizados.

In [10]:
df_1 = df
df_1

,movieId,title,genres,userId,rating,tag
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,336,4.0,pixar
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,474,4.0,pixar
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,567,3.5,fun
3,2,Jumanji (1995),Adventure|Children|Fantasy,62,4.0,fantasy
4,2,Jumanji (1995),Adventure|Children|Fantasy,62,4.0,magic board game
...,...,...,...,...,...,...
3471,187595,Solo: A Star Wars Story (2018),Action|Adventure|Children|Sci-Fi,62,4.0,star wars
3472,193565,Gintama: The Movie (2010),Action|Animation|Comedy|Sci-Fi,184,3.5,anime
3473,193565,Gintama: The Movie (2010),Action|Animation|Comedy|Sci-Fi,184,3.5,comedy
3474,193565,Gintama: The Movie (2010),Action|Animation|Comedy|Sci-Fi,184,3.5,gintama


In [11]:
# Calcularemos el numero de votos
num_votos = df_1.groupby('movieId').size().reset_index(name='numVotos')
#num_votos

# Unimos numVotos en el df original 
df_1 = pd.merge(df_1, num_votos, on='movieId')
df_1

,movieId,title,genres,userId,rating,tag,numVotos
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,336,4.0,pixar,3
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,474,4.0,pixar,3
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,567,3.5,fun,3
3,2,Jumanji (1995),Adventure|Children|Fantasy,62,4.0,fantasy,4
4,2,Jumanji (1995),Adventure|Children|Fantasy,62,4.0,magic board game,4
...,...,...,...,...,...,...,...
3471,187595,Solo: A Star Wars Story (2018),Action|Adventure|Children|Sci-Fi,62,4.0,star wars,2
3472,193565,Gintama: The Movie (2010),Action|Animation|Comedy|Sci-Fi,184,3.5,anime,4
3473,193565,Gintama: The Movie (2010),Action|Animation|Comedy|Sci-Fi,184,3.5,comedy,4
3474,193565,Gintama: The Movie (2010),Action|Animation|Comedy|Sci-Fi,184,3.5,gintama,4


In [12]:
# Calculamos el promedio de de puntaje de cada pelicula
avg_ratings = df_1.groupby('movieId')['rating'].mean().reset_index(name='avgRating')
#avg_ratings
# Unimos avgRating al DataFrame original 
df_1 = pd.merge(df_1, avg_ratings, on='movieId')

In [13]:
df_1.drop_duplicates(subset = ['movieId', 'title', 'avgRating', 'numVotos'], inplace = True)
df_1

,movieId,title,genres,userId,rating,tag,numVotos,avgRating
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,336,4.0,pixar,3,3.833333
3,2,Jumanji (1995),Adventure|Children|Fantasy,62,4.0,fantasy,4,3.750000
7,3,Grumpier Old Men (1995),Comedy|Romance,289,2.5,moldy,2,2.500000
9,5,Father of the Bride Part II (1995),Comedy,474,1.5,pregnancy,2,1.500000
11,7,Sabrina (1995),Comedy|Romance,474,3.0,remake,1,3.000000
...,...,...,...,...,...,...,...,...
3461,183611,Game Night (2018),Action|Comedy|Crime|Horror,62,4.0,Comedy,3,4.000000
3464,184471,Tomb Raider (2018),Action|Adventure|Fantasy,62,3.5,adventure,3,3.500000
3467,187593,Deadpool 2 (2018),Action|Comedy|Sci-Fi,62,4.0,Josh Brolin,3,4.000000
3470,187595,Solo: A Star Wars Story (2018),Action|Adventure|Children|Sci-Fi,62,4.0,Emilia Clarke,2,4.000000


In [18]:


import statistics

# Creamos funcion para hallar la metrica de ponderada
def calculate_weighted_score(avgRating, num_votes, C, m):
    return (num_votes * avgRating + m * C) / (num_votes + m)

# Calculamos promedio del rating global (C)
average_rating = statistics.mean(df_1['avgRating'])
print('The average rating across all movies is:', average_rating)

# Calculamos promedio de numero de votos (m)
avg_num_votes = statistics.mean(df_1['numVotos'])  # Use the average number of votes for threshold
print('The average number of votes is:', avg_num_votes)

# Create a new column 'score' for the weighted average rating using 'avgRating' and 'numVotes'
df_1['score'] = df_1.apply(lambda row: calculate_weighted_score(row['avgRating'], row['numVotos'], average_rating, avg_num_votes), axis=1)

# Display the DataFrame with the calculated weighted score
df_1[['movieId', 'title', 'avgRating', 'numVotos', 'score']].head()

The average rating across all movies is: 3.7323364168313313
The average number of votes is: 2.3743169398907105


,movieId,title,avgRating,numVotos,score
0,1,Toy Story (1995),3.833333,3,3.788714
3,2,Jumanji (1995),3.750000,4,3.743421
7,3,Grumpier Old Men (1995),2.500000,2,3.168895
9,5,Father of the Bride Part II (1995),1.500000,2,2.711680
11,7,Sabrina (1995),3.000000,1,3.515304


In [20]:
df_top = df_1.sort_values(by='score', ascending=False).head(5)
df_top # df con el top 5 peliculas con mayor score segun el numero de votaciones y las votaciones (ranking)

,movieId,title,genres,userId,rating,tag,numVotos,avgRating,score
199,296,Pulp Fiction (1994),Comedy|Crime|Drama|Thriller,103,5.0,good dialogue,181,4.983425,4.967226
1337,2959,Fight Club (1999),Action|Crime|Drama|Thriller,424,4.5,dark comedy,54,4.944444,4.893394
604,924,2001: A Space Odyssey (1968),Adventure|Drama|Sci-Fi,474,4.0,Hal,41,4.951220,4.884498
998,1732,"Big Lebowski, The (1998)",Comedy|Crime,474,3.5,Coen Brothers,32,4.953125,4.868802
164,293,Léon: The Professional (a.k.a. The Professiona...,Action|Crime|Drama|Thriller,166,4.5,assassin,35,4.928571,4.852577


# Recomendacion basada en contenido
La recomendación basada en contenido, se enfoca en los atributos de un item y del perfil de  usuario, recomienda peliculas o productos al perfil de usuario al que mas se ajusten. En este caso podriamos recomendar una pelicula ya sea por el genero, el reparto, o alguna palabra clave que se ajusten a los gustos del usuario. Sin embargo la diversidad de recomendacion podria ser limitada, ya que podriamos recomendar items ya conocidos por el usuario esto hace que este no descubra nuevos items o peliculas que se asemenjen a su perfil.

Queremos calcular la similitud del coseno basándonos en varias características. A continuación, crearemos una columna de características para recopilar las columnas que queremos recomendar a los usuarios. El cálculo se basará en el tipo, los géneros, el país de origen, el idioma, la trama, el resumen y el reparto.

In [24]:
# Crearemos un DF que contenga las columnas necesarias para el analisis

df_2 = df_1[['movieId', 'title', 'userId', 'avgRating', 'numVotos', 'score', 'genres', 'tag']]
df_2.reset_index(drop=True, inplace= True)
df_2.head()

,movieId,title,userId,avgRating,numVotos,score,genres,tag
0,1,Toy Story (1995),336,3.833333,3,3.788714,Adventure|Animation|Children|Comedy|Fantasy,pixar
1,2,Jumanji (1995),62,3.750000,4,3.743421,Adventure|Children|Fantasy,fantasy
2,3,Grumpier Old Men (1995),289,2.500000,2,3.168895,Comedy|Romance,moldy
3,5,Father of the Bride Part II (1995),474,1.500000,2,2.711680,Comedy,pregnancy
4,7,Sabrina (1995),474,3.000000,1,3.515304,Comedy|Romance,remake


In [25]:
# Preparamos los datos de la columna "genres" remplazando "|" con espacios y combinar la columna tag
df_2['features'] = df_2['genres'].str.replace('|', ' ') + ' ' + df_2['tag'].fillna('')
df_2

,movieId,title,userId,avgRating,numVotos,score,genres,tag,features
0,1,Toy Story (1995),336,3.833333,3,3.788714,Adventure|Animation|Children|Comedy|Fantasy,pixar,Adventure Animation Children Comedy Fantasy pixar
1,2,Jumanji (1995),62,3.750000,4,3.743421,Adventure|Children|Fantasy,fantasy,Adventure Children Fantasy fantasy
2,3,Grumpier Old Men (1995),289,2.500000,2,3.168895,Comedy|Romance,moldy,Comedy Romance moldy
3,5,Father of the Bride Part II (1995),474,1.500000,2,2.711680,Comedy,pregnancy,Comedy pregnancy
4,7,Sabrina (1995),474,3.000000,1,3.515304,Comedy|Romance,remake,Comedy Romance remake
...,...,...,...,...,...,...,...,...,...
1459,183611,Game Night (2018),62,4.000000,3,3.881749,Action|Comedy|Crime|Horror,Comedy,Action Comedy Crime Horror Comedy
1460,184471,Tomb Raider (2018),62,3.500000,3,3.602644,Action|Adventure|Fantasy,adventure,Action Adventure Fantasy adventure
1461,187593,Deadpool 2 (2018),62,4.000000,3,3.881749,Action|Comedy|Sci-Fi,Josh Brolin,Action Comedy Sci-Fi Josh Brolin
1462,187595,Solo: A Star Wars Story (2018),62,4.000000,2,3.854716,Action|Adventure|Children|Sci-Fi,Emilia Clarke,Action Adventure Children Sci-Fi Emilia Clarke


Ahora, vectoricemos la columna 'features' con el vectorizador TF-IDF. El vectorizador de Frecuencia de Término-Frecuencia Inversa de Documento (TF-IDF) se utiliza para transformar texto en representaciones numéricas. Evalúa la importancia de una palabra en un documento en relación con un conjunto de documentos, considerando tanto su frecuencia dentro de un documento específico (TF) como su rareza en todos los documentos (IDF).

In [26]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(stop_words='english')

# creamos vector de caracteristicas a partir de la columna 'features'
X = vectorizer.fit_transform(df_2['features'])

In [36]:
X.astype

<bound method _data_matrix.astype of <1464x852 sparse matrix of type '<class 'numpy.float64'>'
	with 5587 stored elements in Compressed Sparse Row format>>

In [41]:
from sklearn.metrics.pairwise import cosine_similarity

#calculamos similitud coseno 
similarity = cosine_similarity(X)

# Recommendation function (including itself as first result)
def recommendation(title, df, similarity, top_n=3):
    try:
        # buscamos indice de la pelicula que coincida con el titien 'title'
        idx = df[df['title'] == title].index[0]
    except IndexError:
        print(f"Movie '{title}' not found in the dataset.")
        return

    # Obtenemos puntaje de similitud de cada pelicula y su indice
    sim_scores = list(enumerate(similarity[idx]))

    # Ordenar de forma desenciente las peliculas segun la similitud coseno 
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # mostrar el top_n de peliculas similares incluyendo la busqueda
    print(f"Movies similar to '{title}' (First movie is itself):")
    for i, (index, score) in enumerate(sim_scores[:top_n+1]):
        movie = df.iloc[index]
        print(f"{i}. {movie['title']} (Similarity Score: {score:.3f})")
        print(f"   Genres: {movie['genres']}")
        print(f"   Tag: {movie['tag']}\n")

# Test de la funcion 'recommendation'
recommendation("Toy Story (1995)", df_2, similarity)

Movies similar to 'Toy Story (1995)' (First movie is itself):
0. Toy Story (1995) (Similarity Score: 1.000)
   Genres: Adventure|Animation|Children|Comedy|Fantasy
   Tag: pixar

1. Bug's Life, A (1998) (Similarity Score: 0.939)
   Genres: Adventure|Animation|Children|Comedy
   Tag: Pixar

2. Toy Story 2 (1999) (Similarity Score: 0.675)
   Genres: Adventure|Animation|Children|Comedy|Fantasy
   Tag: animation

3. Sintel (2010) (Similarity Score: 0.583)
   Genres: Animation|Fantasy
   Tag: adventure



In [44]:
recommendation("Solo: A Star Wars Story (2018)", df_2, similarity)

Movies similar to 'Solo: A Star Wars Story (2018)' (First movie is itself):
0. Solo: A Star Wars Story (2018) (Similarity Score: 1.000)
   Genres: Action|Adventure|Children|Sci-Fi
   Tag: Emilia Clarke

1. X-Men (2000) (Similarity Score: 0.451)
   Genres: Action|Adventure|Sci-Fi
   Tag: action

2. Batman v Superman: Dawn of Justice (2016) (Similarity Score: 0.412)
   Genres: Action|Adventure|Fantasy|Sci-Fi
   Tag: action

3. Jumanji: Welcome to the Jungle (2017) (Similarity Score: 0.403)
   Genres: Action|Adventure|Children
   Tag: Action



# Filtros colavorativos
Los filtros colavorativos es una tecnica de sistemas de recomendacion que hace predicciones automaticas sobre las preferencias del usuario por recopilacion de gustos o preferencias de otros usuarios. Esta tecnica se basa en la premisa de que si ciertos usuarios coincidieron con ciertos elementos en el pasado es probable que coincidan en elementos similares en el futuro.

Existen dos enfoques principales para el filtrado colaborativo:

* Filtrado colaborativo basado en el usuario: Este método identifica a los usuarios con preferencias similares y recomienda elementos que les han gustado. En otras palabras, un usuario recibe recomendaciones basadas en las preferencias de usuarios que históricamente han calificado elementos de forma similar.

* Filtrado colaborativo basado en elementos: En este método, se recomiendan elementos similares a los que el usuario ha calificado o valorado positivamente en el pasado. El sistema identifica los elementos que suelen tener calificaciones similares entre los usuarios y sugiere elementos que comparten estos patrones.


In [45]:
# Pivot user-item matrix from ratings
user_rating_matrix = rating_df.pivot(index="movieId", columns="userId", values="rating")

# fill na with 0
user_rating_matrix = user_rating_matrix.fillna(0)

user_rating_matrix.head()

userId,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
movieId,,,,,,,,,,,,,,,,,,,,,
1,4.0,0.0,0.0,0.0,4.0,0.0,4.5,0.0,0.0,0.0,...,4.0,0.0,4.0,3.0,4.0,2.5,4.0,2.5,3.0,5.0
2,0.0,0.0,0.0,0.0,0.0,4.0,0.0,4.0,0.0,0.0,...,0.0,4.0,0.0,5.0,3.5,0.0,0.0,2.0,0.0,0.0
3,4.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0


En esta sección, utilizaremos un clasificador NearestNeighbors y lo utilizaremos en función de la métrica de similitud de coseno.

In [46]:
from sklearn.neighbors import NearestNeighbors

rec = NearestNeighbors(metric = 'cosine')
rec.fit(user_rating_matrix)

NearestNeighbors(metric='cosine')

Por último, aquí está nuestra función para obtener 5 elementos recomendados en función de una película vista anteriormente.

In [47]:
# Function to get movie recommendations based on a title
def get_recommendations(title):
    # Get movie details
    movie = df_2[df_2['title'] == title]
    
    if movie.empty:
        print(f"Movie '{title}' not found in dataset.")
        return None
    
    movie_id = int(movie['movieId'])
    
    # Get the index of the movie in the user-item matrix
    try:
        user_index = user_rating_matrix.index.get_loc(movie_id)
    except KeyError:
        print(f"Movie ID {movie_id} not found in the user rating matrix.")
        return None
    
    # Get the user ratings for the movie
    user_ratings = user_rating_matrix.iloc[user_index]
    
    # Reshape the ratings to be a single sample (1, -1)
    reshaped_df = user_ratings.values.reshape(1, -1)
    
    # Find the nearest neighbors (similar movies)
    distances, indices = rec.kneighbors(reshaped_df, n_neighbors=15)
    
    # Get the movieIds of the nearest neighbors (excluding the first, which is the queried movie itself)
    nearest_idx = user_rating_matrix.iloc[indices[0]].index[1:]
    
    # Get the movie details for the nearest neighbors
    nearest_neighbors = pd.DataFrame({'movieId': nearest_idx})
    result = pd.merge(nearest_neighbors, df_2, on='movieId', how='left')
    
    # Return the top recommendations
    return result[['title', 'avgRating', 'genres']].head()

# Test the recommendation function
get_recommendations('Toy Story (1995)')

,title,avgRating,genres
0,Toy Story 2 (1999),3.125000,Adventure|Animation|Children|Comedy|Fantasy
1,Jurassic Park (1993),4.500000,Action|Adventure|Sci-Fi|Thriller
2,Independence Day (a.k.a. ID4) (1996),4.000000,Action|Adventure|Sci-Fi|Thriller
3,Star Wars: Episode IV - A New Hope (1977),4.527778,Action|Adventure|Sci-Fi
4,Forrest Gump (1994),3.666667,Comedy|Drama|Romance|War


In [48]:
get_recommendations('Jurassic Park (1993)')

,title,avgRating,genres
0,Terminator 2: Judgment Day (1991),2.625000,Action|Sci-Fi
1,Forrest Gump (1994),3.666667,Comedy|Drama|Romance|War
2,Braveheart (1995),4.350000,Action|Drama|War
3,"Fugitive, The (1993)",5.000000,Thriller
4,Speed (1994),4.000000,Action|Romance|Thriller


:)

# Agradecimiento a los autores de este ejercicio y a los colaboradores:

# Authors

Lucy Xu

Ricky Shi

# Contributors
Hailey Quach

Copyright © 2024 IBM Corporation. All rights reserved.